# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [12]:
# Import libraries
import numpy as np
import pandas as pd

In [13]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

#print sum(np.where(student_data['passed'][:].values=="no",1,0))

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [14]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]+0.0
n_features = student_data.shape[1]-1.0
n_passed = sum(np.where(student_data['passed'][:].values=="yes",1,0))+0.0
n_failed = sum(np.where(student_data['passed'][:].values=="no",1,0))+0.0
grad_rate = (n_passed+0.0)/(n_students + 0.0)*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)


Total number of students: 395.0
Number of students who passed: 265.0
Number of students who failed: 130.0
Number of features: 30.0
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [15]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows
print y_all.head()

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [16]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))
#print X_all


Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [17]:
# First, decide how many training vs test samples you want
from sklearn import cross_validation

num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train
myRandom = np.random.random_integers(0, 500)
# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
#I received the following from the pandas and numpy websites
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, train_size=num_train, random_state=myRandom, stratify = y_all)
SSS = cross_validation.StratifiedShuffleSplit(y_train, test_size=num_test,n_iter=50, random_state=myRandom+22)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data


Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [18]:
# Train a model
import time


#I wanted to have some fun and test most of the models I learned about
from sklearn import svm
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import grid_search
from sklearn import metrics
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\n" +str(clf.__class__.__name__) + " " + str(X_train.shape[0]) + " size Training time (millisecs): {:.3f}".format((end - start)*1000)

#my custom gridsearch function using Stratified Shuffle Split for Cross Validation - generated from the training set
# so don't use this when you change training set size - iterate through them that is
def run_CV(svr, X_train, y_train,parameters,cvnum, err_score, mScorer):
    print "Running GridSearchCV {}...".format(svr.__class__.__name__)
    start = time.time()
    clf = grid_search.GridSearchCV(svr, parameters,cv = cvnum, error_score = err_score, scoring=mScorer)
    clf.fit(X_train, y_train)
    print 'Best estimator:'
    print clf.best_estimator_
    newclf = clf.best_estimator_
    end = time.time()
    print "Done!\nGridSearchCV time (millisecs): {:.3f}".format((end - start)*1000)
    return newclf


fone_scorer = make_scorer(metrics.f1_score,greater_is_better = True, pos_label=None,average='weighted')


#easy way to change the F1 score parameters instead of having to change many lines
def myF1Score(test,pred):
    return metrics.f1_score(test, pred, pos_label=None,average='weighted')



# TODO: Choose a model, import it and instantiate an object


#Decision Tree
clf = tree.DecisionTreeClassifier()

max_depths = 10 
parameters = { 'max_features': ('auto',None), 'max_depth':range(1,max_depths),'presort':(True,False)}

#After running the code quite a number of times GridSearchCV returned this model, which is much better than anything
#I have seen thus far (anything where max_depth is not none), and which points out that the selection of the initial training set
#plays a huge part in how we train our model
#It probab;y implies that our data set is not really large enough to get a good model
#DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
#            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
#            min_samples_split=2, min_weight_fraction_leaf=0.0,
#            presort=True, random_state=None, splitter='best')
#F1_score:0.941176470588


#Fit model to training data
#train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
#y_pred = clf.predict(X_test)
#print "F1_score:",str(myF1Score(y_test.values, y_pred))

def PrintResults(y_test,y_pred):
#     print 'We made ' + str(y_test.shape[0]) + ' predictions'
#     print str(sum(y_test==y_pred)) +' of ' + str((y_test.shape[0])) + ' predictions were correct'
#     print str(sum(y_test!=y_pred)) +' of ' + str((y_test.shape[0])) + ' predictions were incorrect'
#     predictedPass = float(y_pred[y_pred == 'yes'].shape[0])
#     predictedFail = float(y_pred[y_pred == 'no'].shape[0])
#     ActualPass = float(y_test[y_test == 'yes'].shape[0])
#     ActualFail = float(y_test[y_test == 'no'].shape[0])
#     TP = float(sum((y_test==y_pred)[y_test == 'yes']==True))
#     TN = float(sum((y_test==y_pred)[y_test == 'no']==True))
#     FP = float(sum((y_test!=y_pred)[y_test == 'no']==True))
#     FN = float(sum((y_test!=y_pred)[y_test == 'yes']==True))
    
#     print 'tp',TP
#     print 'tn',TN
#     print 'fn',FN
#     print 'fp',FP
    
    
#     print 'We predicted that ' + str(predictedPass) + ' students will pass and in reality ' + str(ActualPass) + ' passed'
#     print 'We predicted that ' + str(predictedFail) + ' students will fail and in reality ' + str(ActualFail) + ' failed'
#     print 'We predicted '+ str(TP) +' of the '+str(ActualPass) + ' students who passed'
#     print 'Of the passes we predicted,  '+ str(FP) +' turned out to be fails'
#     print 'Thus we missed '+str(sum((y_test==y_pred)[y_pred == 'yes']==False))+' interventions'
#     print 'We predicted '+ str(TN) +' of the '+str(ActualFail) + ' students who failed'
#     print 'Of the fails we predicted,  '+ str(sum((y_test==y_pred)[y_pred == 'no']==False)) +' turned out to be passes'
#     print 'Thus we intervened '+str(sum((y_test==y_pred)[y_pred == 'no']==False))+' times where we did not have to intervene'
    print 'Metrics precision',metrics.precision_score(y_test,y_pred,pos_label=None,average='weighted')
    print 'Metrics recall',metrics.recall_score(y_test,y_pred,pos_label=None,average='weighted')
    print 'precision_recall_fscore_support ', metrics.precision_recall_fscore_support(y_test,y_pred,pos_label=None,average='weighted')
#     dcRecall = float(TP/(TP+FN))
#     dcPrecision = float(TP/(TP+FP))
    
#     print 'Precision is ' ,dcPrecision
#     print 'Recall is ',dcRecall
#     print 'My f1 score is '+str(2* ((dcPrecision*dcRecall)/(dcPrecision+dcRecall)))


#some additional code I added
print 'Starting GridSearch Cross Validation'
clf = run_CV( clf,X_train, y_train,parameters,SSS,0,fone_scorer)
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it
y_pred = clf.predict(X_test)
print str(clf.__class__.__name__) + " test F1_score:" + str(myF1Score(y_test.values, y_pred))

PrintResults(y_test.values,y_pred)



Starting GridSearch Cross Validation
Running GridSearchCV DecisionTreeClassifier...
Best estimator:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=True, random_state=None, splitter='best')
Done!
GridSearchCV time (millisecs): 8274.000
Training DecisionTreeClassifier...
Done!
DecisionTreeClassifier 300 size Training time (millisecs): 1.000
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=True, random_state=None, splitter='best')
DecisionTreeClassifier test F1_score:0.718875054755
Metrics precision 0.724105263158
Metrics recall 0.736842105263
precision_recall_fscore_support  (0.72410526315789481, 0.73684210526315785, 0.718875054754957

In [19]:
# Predict on training set and compute F1 score


def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (millisecs): {:.3f}".format((end - start)*1000)
    return myF1Score(target.values, y_pred)

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (millisecs): 1.000
F1 score for training set: 0.700395738204


In [20]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (millisecs): 1.000
F1 score for test set: 0.718875054755


In [21]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    clf = run_CV( clf,X_train, y_train,parameters,SSS,0,fone_scorer)
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

def train_predict_noCV(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
    
    
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant


print "------------------------------------------"
print 'Running GridSearchCV across multiple training size sets'

ranges = (100,200,300)
for i in ranges:
    X_train1 = X_all.sample(n=i,frac=None,replace=False,weights=None,random_state=myRandom,axis=None)
    y_train1 = y_all.sample(n=i,frac=None,replace=False,weights=None,random_state=myRandom,axis=None)
        
    clf = tree.DecisionTreeClassifier()
    #SSS will throw an errorif we change train sizes
    train_predict_noCV(clf,X_train[0:i],y_train[0:i],X_test,y_test)

   


------------------------------------------
Running GridSearchCV across multiple training size sets
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
DecisionTreeClassifier 100 size Training time (millisecs): 1.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (millisecs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (millisecs): 0.000
F1 score for test set: 0.639532163743
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
DecisionTreeClassifier 200 size Training time (millisecs): 3.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (millisecs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (millisecs): 0.000
F1 score for test set: 0.705263157895
------------------------------------------
Training set

In [22]:
# TODO: Train and predict using two other models
#I decided to automate the training set size and model testing and combine it with GridSearchCV
#The below code does that
#I played around with the different parameteres for each model

ModelDict = {}
ParamDict = {}

#Decision Tree
Decclf = tree.DecisionTreeClassifier()
max_depths = 30 #I don't see much point going past 30
Decclf_params = { 'max_features': ('auto',None), 'max_depth':range(1,max_depths),'presort':(True,False)}
ModelDict['Dec'] = Decclf
ParamDict['Dec'] = Decclf_params

#Multinomial Naive Bayes
NBclf = MultinomialNB()
NBclf_params = {'alpha':(0,1.0),  'fit_prior':(True,False)}
ModelDict['NB'] = NBclf
ParamDict['NB'] = NBclf_params

#Stochastic Gradient Descent
#need to figure ou these parameters
Stochclf = SGDClassifier(loss="log",penalty="l2")
Stochclf_params = {'shuffle':(True,False),'warm_start':(True,False),'n_iter':range(5,30,5) }

ModelDict['Stoch'] = Stochclf
ParamDict['Stoch'] = Stochclf_params

#K-NN classifier
KNNclf = KNeighborsClassifier()
KNNclf_params = {'n_neighbors':range(1,30,2),'weights':('uniform','distance'),'algorithm':('auto','ball_tree','kd_tree','brute'), }
#class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1, **kwargs)
ModelDict['KNN'] = KNNclf
ParamDict['KNN'] = KNNclf_params

#SVM svc
SVMclf = svm.SVC()
SVMclf_params = {'C':(1,2,3,4,5,10),'kernel':('linear', 'rbf','sigmoid','poly'), 'shrinking':(True,False)}
#SVMclf_params = {'C':(5,4),'kernel':('rbf','rbf'), 'shrinking':(True,True)}
ModelDict['SVM'] = SVMclf
ParamDict['SVM'] = SVMclf_params

#Gaussian Naive Bayes
Gaussclf = GaussianNB()
Gaussclf_params = {}
ModelDict['GS'] = Gaussclf
ParamDict['GS'] = Gaussclf_params

ranges = (100,200,300)

models = ('Dec','NB','Stoch','KNN','SVM','GS') 
results = []


print "------------------------------------------"
print "--Get Basic F1 Scores---------------------"
#Just some code to get the basic model F1 score
for i in models:
    print "------------------------------------------"
    for k in ranges:
        print "------------------------------------------"
        print "Training set size: {}".format(k)
        clf = ModelDict[i]
        X_train1 = X_all.sample(n=k,frac=None,replace=False,weights=None,random_state=myRandom,axis=None)
        y_train1 = y_all.sample(n=k,frac=None,replace=False,weights=None,random_state=myRandom,axis=None)
        
        train_classifier(clf, X_train1, y_train1) 
        print 'Basic fitted model'
        print clf
        y_pred = clf.predict(X_train1)
        print str(clf.__class__.__name__) + " " + str(k) + " size train F1_score:" + str(myF1Score(y_train1.values, y_pred))
        start = time.time()
        y_pred = clf.predict(X_test)
        end = time.time()
        print "Done!\n" +str(clf.__class__.__name__) + " " + str(k) + " size Prediction time (millisecs): {:.3f}".format((end - start)*1000)
        print str(clf.__class__.__name__) + " " + str(k) + " size test F1_score:" + str(myF1Score(y_test.values, y_pred))
    print "------------------------------------------"


    
    

------------------------------------------
--Get Basic F1 Scores---------------------
------------------------------------------
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
DecisionTreeClassifier 100 size Training time (millisecs): 1.000
Basic fitted model
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
DecisionTreeClassifier 100 size train F1_score:1.0
Done!
DecisionTreeClassifier 100 size Prediction time (millisecs): 0.000
DecisionTreeClassifier 100 size test F1_score:0.754511715957
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
DecisionTreeClassifier 200 size Training time (millisecs): 2.000
Basic fitted model
DecisionTreeClassifier(class_w

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [23]:


#Decision Tree Classifier


max_depths = 10


clf_params= {          'clf__criterion':['entropy','gini'],
                       'clf__max_depth': [1,10,None],
                        'clf__presort': [True,False],
                        'clf__class_weight': [{'no': 15, 'yes': 1},
                                               {'no': 12, 'yes': 1},
                                               {'no': 10, 'yes': 1},
                                               {'no': 8, 'yes': 1},
                                               {'no': 4, 'yes': 1},
                                               {'no': 1, 'yes': 15},
                                               {'no': 1, 'yes': 12},
                                               {'no': 1, 'yes': 10},
                                               {'no': 1, 'yes': 8},
                                               {'no': 1, 'yes': 4},
                                               'balanced']
                      } # This is the list of parameters and range of values I want to search.

# clf_params= {
#                        'clf__max_features': ['auto',None],
#                        'clf__max_depth': range(1,max_depths),
#                         'clf__presort': [True,False],
#                         'clf__class_weight': [{'no': 12, 'yes': 1},
#                                                {'no': 10, 'yes': 1},
#                                                {'no': 8, 'yes': 1},
#                                                {'no': 15, 'yes': 1},
#                                                {'no': 4, 'yes': 1},
#                                                'balanced', None]
#                       }


start = time.time()

print "Done!\nStart time: {:.3f}".format(( start))
#For this Pipeline:  http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipe = Pipeline(steps=[ ('clf', tree.DecisionTreeClassifier())])

a_grid_search = grid_search.GridSearchCV(pipe, param_grid = clf_params,cv = SSS,iid=False, scoring = fone_scorer)
a_grid_search.fit(X_train,y_train)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
# pick a winner
best_clf = a_grid_search.best_estimator_  # best_estimator_ helps to identify the best classifier.
print best_clf

best_clf.fit(X_train,y_train)
y_pred = best_clf.predict(X_train)
print str(best_clf.__class__.__name__) + " train F1_score:" + str(myF1Score(y_train.values, y_pred))
start = time.time()
y_pred = best_clf.predict(X_test)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
print str(best_clf.__class__.__name__) + " test F1_score:" + str(myF1Score(y_test.values, y_pred))
print "------------------------------------------"
PrintResults(y_test.values,y_pred)




Done!
Start time: 1457298984.995
Done!
Prediction time (secs): 101.866
Pipeline(steps=[('clf', DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=True, random_state=None, splitter='best'))])
Pipeline train F1_score:0.700395738204
Done!
Prediction time (secs): 0.000
Pipeline test F1_score:0.718875054755
------------------------------------------
Metrics precision 0.724105263158
Metrics recall 0.736842105263
precision_recall_fscore_support  (0.72410526315789481, 0.73684210526315785, 0.71887505475495761, None)


In [ ]:
#Here we train KNN


clf_params= {
                       'clf__n_neighbors': [5, 10, 15, 20, 25, 30],
                       'clf__weights': ['uniform','distance'],
                       'clf__leaf_size': [3,5,8,10,15,20,30,40,50,60,70,80],
                       'clf__metric': ['minkowski','euclidean','manhattan','chebyshev']
                      } # This is the list of parameters and range of values I want to search.

start = time.time()

print "Done!\nStart time: {:.3f}".format(( start))
#For this Pipeline:  http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipe = Pipeline(steps=[('normalize', preprocessing.StandardScaler()), ('clf', KNeighborsClassifier())])# Note KNN calls for standardized features, so Scale is used prior to fit and tune KNN

a_grid_search = grid_search.GridSearchCV(pipe, param_grid = clf_params,cv = SSS,iid=False, scoring = fone_scorer)
a_grid_search.fit(X_train,y_train)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
# pick a winner
best_clf = a_grid_search.best_estimator_  # best_estimator_ helps to identify the best classifier.
print best_clf

best_clf.fit(X_train,y_train)
y_pred = best_clf.predict(X_train)
print str(best_clf.__class__.__name__) + " train F1_score:" + str(myF1Score(y_train.values, y_pred))
start = time.time()
y_pred = best_clf.predict(X_test)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
print str(best_clf.__class__.__name__) + " test F1_score:" + str(myF1Score(y_test.values, y_pred))
print "------------------------------------------"
PrintResults(y_test.values,y_pred)


In [ ]:
#Here we train SVC

clf_params= {
                       'clf__C': [ 1e-2, 1e-1, 1, 10, 1e2],
                       'clf__gamma': [0.0,'auto'],
                       'clf__kernel': ['linear', 'poly', 'rbf'],
                       'clf__tol': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
                      } # This is the list of parameters and range of values I want to search.

start = time.time()

print "Done!\nStart time: {:.3f}".format(( start))
#For this Pipeline:  http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipe = Pipeline(steps=[('normalize', preprocessing.StandardScaler()), ('clf', svm.SVC())])# Note SVM calls for standardized features, so Scale is used prior to fit and tune SVM

a_grid_search = grid_search.GridSearchCV(pipe, param_grid = clf_params,cv = SSS,iid=False, scoring = fone_scorer)
a_grid_search.fit(X_train,y_train)
end = time.time()
print "Done!\nPrediction time (millisecs): {:.3f}".format((end - start)*1000)
# pick a winner
best_clf = a_grid_search.best_estimator_  # best_estimator_ helps to identify the best classifier.
print best_clf

best_clf.fit(X_train,y_train)
y_pred = best_clf.predict(X_train)
print str(best_clf.__class__.__name__) + " train F1_score:" + str(myF1Score(y_train.values, y_pred))
start = time.time()
y_pred = best_clf.predict(X_test)
end = time.time()
print "Done!\nPrediction time (millisecs): {:.3f}".format((end - start)*1000)
print str(best_clf.__class__.__name__) + " test F1_score:" + str(myF1Score(y_test.values, y_pred))
print "------------------------------------------"
PrintResults(y_test.values,y_pred)

In [ ]:
#Here we train Multinomial NB
#Multinomial Naive Bayes

clf_params= {
                       'clf__alpha': [0,0.2,0.4,0.6,0.8,1.0,2,4,6,8,10],
                       'clf__fit_prior': [False,True],

                      } # This is the list of parameters and range of values I want to search.

start = time.time()

print "Done!\nStart time: {:.3f}".format(( start))
#For this Pipeline:  http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipe = Pipeline(steps=[ ('clf', MultinomialNB())])

a_grid_search = grid_search.GridSearchCV(pipe, param_grid = clf_params,cv = SSS,iid=False, scoring = fone_scorer)
a_grid_search.fit(X_train,y_train)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
# pick a winner
best_clf = a_grid_search.best_estimator_  # best_estimator_ helps to identify the best classifier.
print best_clf

best_clf.fit(X_train,y_train)
y_pred = best_clf.predict(X_train)
print str(best_clf.__class__.__name__) + " train F1_score:" + str(myF1Score(y_train.values, y_pred))
start = time.time()
y_pred = best_clf.predict(X_test)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
print str(best_clf.__class__.__name__) + " test F1_score:" + str(myF1Score(y_test.values, y_pred))
print "------------------------------------------"
PrintResults(y_test.values,y_pred)

In [ ]:
#Here we train Gaussian NB
#Gaussian Naive Bayes

clf_params= {
                   
                      } # This is the list of parameters and range of values I want to search.

start = time.time()

print "Done!\nStart time: {:.3f}".format(( start))
#For this Pipeline:  http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipe = Pipeline(steps=[ ('clf', GaussianNB())])

a_grid_search = grid_search.GridSearchCV(pipe, param_grid = clf_params,cv = SSS,iid=False, scoring = fone_scorer)
a_grid_search.fit(X_train,y_train)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
# pick a winner
best_clf = a_grid_search.best_estimator_  # best_estimator_ helps to identify the best classifier.
print best_clf

best_clf.fit(X_train,y_train)
y_pred = best_clf.predict(X_train)
print str(best_clf.__class__.__name__) + " train F1_score:" + str(myF1Score(y_train.values, y_pred))
start = time.time()
y_pred = best_clf.predict(X_test)
end = time.time()
print "Done!\nPrediction time (secs): {:.3f}".format((end - start))
print str(best_clf.__class__.__name__) + " test F1_score:" + str(myF1Score(y_test.values, y_pred))
print "------------------------------------------"
PrintResults(y_test.values,y_pred)